In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np

"""
このnotebookがbaseフォルダを読めるようになるための処理
"""

root_rel = '../../'

# 相対パスを絶対パスに変換してsys.pathに追加
root_abs = os.path.abspath(root_rel)
if root_abs not in sys.path:
    sys.path.append(root_abs)
print(root_abs)


# テストデータのpath

In [ ]:
# テストデータのpath
dist_path = "rotated_OIbDia08_6_dist(50).csv"
calib_path = "rotated_OIbDia08_6_calib.csv"
spe_path = "OIbDia08_ 06.spe"

nxs_path = "OIbDia08_6_00000.nxs"
poni_path = "Dioptas_lambda_MgS400_CeO2_240511.poni"

# HDF5ファイルの設定

In [ ]:
import h5py

In [ ]:
def show_hdf5_hierarchy(f, indent=0, path=""):
    for key in f.keys():
        print(" " * indent + "/" + key)  # グループ名を出力
        current_path = path + "/" + key
        if isinstance(f[key], h5py.Group): # もしkey先がgroupの場合
            path += "/"
            show_hdf5_hierarchy(f[key], indent + 4, path=current_path)  # グループ内のさらに深い階層を出力
        else:
            print(" " * indent + f" --> path:  \"{current_path}\"")
            print(" " * indent + f"       L {type(f[key])}")
            dataset = f[key]
            
            if dataset.shape == (): # スカラー(単一値)の場合
                value = str(dataset[()]) # スカラーの場合の読み取り
            else:
                value = str(f.get(key)[0])
                
            if len(value) > 20:
                print(" " * indent + f"       L {value[:20]}")
            else:
                print(" " * indent + f"       L {value}")



In [ ]:
# calibのテストで作ったやつ
with h5py.File("CalibData.hdf5", "r") as f: 
    show_hdf5_hierarchy(f)

# 全部入れてみる

In [ ]:
# これに入れる
hdf5_name = "hdf5_test.hdf5"

In [ ]:
from model.Temperature import (
    Laser,
    Spe,
    T_calib,
    T_dist
)

from model.XRD import Nxs

## Speデータ

In [ ]:
spe = Spe.SpeFile(filepath=spe_path)

In [ ]:
dir(spe)

In [ ]:
# 階層pathを作る
data_layer = "data/" # 一応冗長にしてる
temperature_layer = data_layer + "temperature/"

In [ ]:
spe_layer = temperature_layer + "spe/"

param_layer = spe_layer + "param/"
arr_layer = spe_layer + "arr/"

# speのparam
spe_path_layer = param_layer + "spe_path"
spe_frame_num_layer = param_layer + "spe_frame_num"
spe_exposure_ms_layer = param_layer + "spe_exposure_ms"
spe_fps_layer = param_layer + "spe_fps"
spe_OD_layer = param_layer + "spe_OD"
spe_date_layer = param_layer + "spe_date"
spe_calibration_date_layer = param_layer + "spe_calibration_date"
# speのarr
spe_img_layer = arr_layer + "spe_img"
spe_wl_arr = arr_layer + "spe_wl_arr"
spe_time_arr = arr_layer + "spe_time_arr"
spe_pixel_arr = arr_layer + "spe_pixel_arr"

In [ ]:
spe.time_arr = np.arange(spe.num_frames) / spe.framerate
spe.pixel_arr = np.arange(spe.roi_height)

In [ ]:
# 書き込み
# これはHDF5モデルに任せる。
with h5py.File(hdf5_name, "w") as f:
    # TODO 今はめんどくさくてやってないが、ここでデータ型を指定しておくべきかもしれない
    # param
    f.create_dataset(spe_path_layer,             data=spe.filename)
    f.create_dataset(spe_frame_num_layer,        data=spe.num_frames)
    f.create_dataset(spe_exposure_ms_layer,      data=spe.exposure_time)
    f.create_dataset(spe_fps_layer,              data=spe.framerate)
    f.create_dataset(spe_OD_layer,               data=spe.OD)
    f.create_dataset(spe_date_layer,             data=spe.date)
    f.create_dataset(spe_calibration_date_layer, data=spe.calibration_date)
    # arr
    f.create_dataset(spe_img_layer, data=spe.img)
    f.create_dataset(spe_wl_arr,    data=spe.x_calibration)
    f.create_dataset(spe_time_arr,  data=spe.time_arr)
    f.create_dataset(spe_pixel_arr, data=spe.time_arr)

In [ ]:
# 確認
with h5py.File(hdf5_name, "r") as f:
    show_hdf5_hierarchy(f)

## calibデータ

In [ ]:
# 確認
with h5py.File(hdf5_name, "r") as f:
    show_hdf5_hierarchy(f)

## distデータ

In [ ]:
# 確認
with h5py.File(hdf5_name, "r") as f:
    show_hdf5_hierarchy(f)


## laserデータ

In [ ]:
# 確認
with h5py.File(hdf5_name, "r") as f:
    show_hdf5_hierarchy(f)
